# SAP HANA Cloud Vector Engine

>[SAP HANA Cloud Vector Engine](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-vector-engine-guide/sap-hana-cloud-sap-hana-database-vector-engine-guide) is a vector store fully integrated into the `SAP HANA Cloud` database.

## Setup

Install the `langchain-hana` external integration package, as well as the other packages used throughout this notebook.

In [8]:
%pip install -qU langchain-hana

^C
Note: you may need to restart the kernel to use updated packages.


### Credentials

Ensure your SAP HANA instance is running. Load your credentials from environment variables and create a connection:

In [1]:
import os
import json
print(os.getenv('AICORE_HOME', 'default_value'))

C:\Users\changgoo.kang\PycharmProjects\ai-core-samples\.aicore


In [2]:
# 환경 변수에서 경로 및 프로파일 읽어오기
aicore_home = os.getenv('AICORE_HOME')
#profile = os.getenv('AICORE_PROFILE', 'default')
config_path = os.path.join(aicore_home, f"config.json")

In [3]:
from ai_core_sdk.ai_core_v2_client import AICoreV2Client
# 구성 파일에서 값 읽어오기
with open(config_path, 'r') as config_file:
    config = json.load(config_file)

# 클라이언트 초기화
ai_core_client = AICoreV2Client(
    base_url=config['AICORE_BASE_URL'],
    auth_url=config['AICORE_AUTH_URL'],
    client_id=config['AICORE_CLIENT_ID'],
    client_secret=config['AICORE_CLIENT_SECRET']
)

print("AI Core client initialized successfully!")

AI Core client initialized successfully!


In [10]:
model_list=['gpt-4o-mini', 'anthropic--claude-3.5-sonnet', 'text-embedding-3-small']

In [5]:
# 환경 변수에서 경로 및 프로파일 읽어오기
aicore_home = os.getenv('AICORE_HOME')
hana_config_path = os.path.join(aicore_home, "hana_config.json")

with open(hana_config_path, 'r') as hana_config_file:
    hana_config = json.load(hana_config_file)

In [6]:

# HANA 연결 설정
my_hana_config = {
    'address': hana_config['HOST'],
    'port': hana_config['PORT'],  # 기본 포트
    'user': hana_config['USER'],
    'password': hana_config['PASSWORD'],
    'encrypt': 'true',
    'sslValidateCertificate': 'false'  # 개발 환경에서는 false, 운영은 true 권장
}

In [7]:
from hdbcli import dbapi
    # DB 연결
    # conn = dbapi.connect(**my_hana_config)
connection = dbapi.connect(
    address=hana_config['HOST'],
    port=hana_config['PORT'],
    user=hana_config['USER'],
    password=hana_config['PASSWORD'],
)
cursor = connection.cursor()

In [2]:



from dotenv import load_dotenv
from hdbcli import dbapi

load_dotenv()
# Use connection settings from the environment
connection = dbapi.connect(
    address=os.environ.get("HANA_DB_ADDRESS"),
    port=os.environ.get("HANA_DB_PORT"),
    user=os.environ.get("HANA_DB_USER"),
    password=os.environ.get("HANA_DB_PASSWORD"),
    autocommit=True,
    sslValidateCertificate=False,
)

Learn more about SAP HANA in [What is SAP HANA?](https://www.sap.com/products/data-cloud/hana/what-is-sap-hana.html).

### Initialization
To initialize a `HanaDB` vector store, you need a database connection and an embedding instance. SAP HANA Cloud Vector Engine supports both external and internal embeddings.

- #### Using External Embeddings

import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs/>

In [9]:
from gen_ai_hub.proxy.langchain.init_models import init_llm
 
llm = init_llm('anthropic--claude-3.5-sonnet', temperature=0.0, max_tokens=256)
response = llm.invoke('What is text embeddings. Write it in 20 words in korean?').content 
print('Response:', response)

Response: 텍스트 임베딩은 단어나 문장을 컴퓨터가 이해할 수 있는 숫자 벡터로 변환하는 기술입니다.


In [11]:
from gen_ai_hub.proxy.langchain.init_models import init_llm
from gen_ai_hub.proxy.langchain.init_models import init_embedding_model

# llm = init_llm('gpt-4o-mini', temperature=0., max_tokens=256)
embeddings = init_embedding_model('text-embedding-3-small')

In [3]:
# | output: false
# | echo: false
# from langchain_openai import OpenAIEmbeddings
# 
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

- #### Using Internal Embeddings

Alternatively, you can compute embeddings directly in SAP HANA using its native `VECTOR_EMBEDDING()` function. To enable this, create an instance of `HanaInternalEmbeddings` with your internal model ID and pass it to `HanaDB`. Note that the `HanaInternalEmbeddings` instance is specifically designed for use with `HanaDB` and is not intended for use with other vector store implementations. For more information about internal embedding, see the [SAP HANA VECTOR_EMBEDDING Function](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-vector-engine-guide/vector-embedding-function-vector).

> **Caution:** Ensure NLP is enabled in your SAP HANA Cloud instance.

In [4]:
from langchain_hana import HanaInternalEmbeddings

embeddings = HanaInternalEmbeddings(internal_embedding_model_id="SAP_NEB.20240715")

Once you have your connection and embedding instance, create the vector store by passing them to `HanaDB` along with a table name for storing vectors:

In [12]:
from langchain_hana import HanaDB

db = HanaDB(
    embedding=embeddings, connection=connection, table_name="STATE_OF_THE_UNION"
)

## Example

Load the sample document "state_of_the_union.txt" and create chunks from it.

In [13]:
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

text_documents = TextLoader(
    "../../how_to/state_of_the_union.txt", encoding="UTF-8"
).load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
text_chunks = text_splitter.split_documents(text_documents)
print(f"Number of document chunks: {len(text_chunks)}")

Number of document chunks: 88


Add the loaded document chunks to the table. For this example, we delete any previous content from the table which might exist from previous runs.

In [14]:
# Delete already existing documents from the table
db.delete(filter={})

# add the loaded document chunks
db.add_documents(text_chunks)

[]

Perform a query to get the two best-matching document chunks from the ones that were added in the previous step.
By default "Cosine Similarity" is used for the search.

In [15]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query, k=2)

for doc in docs:
    print("-" * 80)
    print(doc.page_content)

--------------------------------------------------------------------------------
One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. 

While it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice.


Query the same content with "Euclidian Distance". The results shoud be the same as with "Cosine Similarity".

In [9]:
from langchain_hana.utils import DistanceStrategy

db = HanaDB(
    embedding=embeddings,
    connection=connection,
    distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE,
    table_name="STATE_OF_THE_UNION",
)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query, k=2)
for doc in docs:
    print("-" * 80)
    print(doc.page_content)

--------------------------------------------------------------------------------
One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. 

While it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice.


## Maximal Marginal Relevance Search (MMR)

`Maximal marginal relevance` optimizes for similarity to query AND diversity among selected documents. The first 20 (fetch_k) items will be retrieved from the DB. The MMR algorithm will then find the best 2 (k) matches.

In [10]:
docs = db.max_marginal_relevance_search(query, k=2, fetch_k=20)
for doc in docs:
    print("-" * 80)
    print(doc.page_content)

--------------------------------------------------------------------------------
One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
Groups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland. 

In this struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkness.” The Ukrainian Ambassador to the United States is here tonight. 

Let each of us here tonight in this Chamber send an unmistakable signal to Ukraine and to the world.


## Creating an HNSW Vector Index

A vector index can significantly speed up top-k nearest neighbor queries for vectors. Users can create a Hierarchical Navigable Small World (HNSW) vector index using the `create_hnsw_index` function.

For more information about creating an index at the database level, please refer to the [official documentation](https://help.sap.com/docs/hana-cloud-database/sap-hana-cloud-sap-hana-database-vector-engine-guide/create-vector-index-statement-data-definition).



In [11]:
# HanaDB instance uses cosine similarity as default:
db_cosine = HanaDB(
    embedding=embeddings, connection=connection, table_name="STATE_OF_THE_UNION"
)

# Attempting to create the HNSW index with default parameters
db_cosine.create_hnsw_index()  # If no other parameters are specified, the default values will be used
# Default values: m=64, ef_construction=128, ef_search=200
# The default index name will be: STATE_OF_THE_UNION_COSINE_SIMILARITY_IDX (verify this naming pattern in HanaDB class)


# Creating a HanaDB instance with L2 distance as the similarity function and defined values
db_l2 = HanaDB(
    embedding=embeddings,
    connection=connection,
    table_name="STATE_OF_THE_UNION",
    distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE,  # Specify L2 distance
)

# This will create an index based on L2 distance strategy.
db_l2.create_hnsw_index(
    index_name="STATE_OF_THE_UNION_L2_index",
    m=100,  # Max number of neighbors per graph node (valid range: 4 to 1000)
    ef_construction=200,  # Max number of candidates during graph construction (valid range: 1 to 100000)
    ef_search=500,  # Min number of candidates during the search (valid range: 1 to 100000)
)

# Use L2 index to perform MMR
docs = db_l2.max_marginal_relevance_search(query, k=2, fetch_k=20)
for doc in docs:
    print("-" * 80)
    print(doc.page_content)

--------------------------------------------------------------------------------
One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.
--------------------------------------------------------------------------------
Groups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland. 

In this struggle as President Zelenskyy said in his speech to the European Parliament “Light will win over darkness.” The Ukrainian Ambassador to the United States is here tonight. 

Let each of us here tonight in this Chamber send an unmistakable signal to Ukraine and to the world.



**Key Points**:
- **Similarity Function**: The similarity function for the index is **cosine similarity** by default. If you want to use a different similarity function (e.g., `L2` distance), you need to specify it when initializing the `HanaDB` instance.
- **Default Parameters**: In the `create_hnsw_index` function, if the user does not provide custom values for parameters like `m`, `ef_construction`, or `ef_search`, the default values (e.g., `m=64`, `ef_construction=128`, `ef_search=200`) will be used automatically. These values ensure the index is created with reasonable performance without requiring user intervention.



## Basic Vectorstore Operations

In [16]:
db = HanaDB(
    connection=connection, embedding=embeddings, table_name="LANGCHAIN_DEMO_BASIC"
)

# Delete already existing documents from the table
db.delete(filter={})

True

We can add simple text documents to the existing table.

In [17]:
docs = [Document(page_content="Some text"), Document(page_content="Other docs")]
db.add_documents(docs)

[]

Add documents with metadata.

In [18]:
docs = [
    Document(
        page_content="foo",
        metadata={"start": 100, "end": 150, "doc_name": "foo.txt", "quality": "bad"},
    ),
    Document(
        page_content="bar",
        metadata={"start": 200, "end": 250, "doc_name": "bar.txt", "quality": "good"},
    ),
]
db.add_documents(docs)

[]

Query documents with specific metadata.

In [19]:
docs = db.similarity_search("foobar", k=2, filter={"quality": "bad"})
# With filtering on "quality"=="bad", only one document should be returned
for doc in docs:
    print("-" * 80)
    print(doc.page_content)
    print(doc.metadata)

Plain SQL Placeholder '?' for value='bad'


--------------------------------------------------------------------------------
foo
{'start': 100, 'end': 150, 'doc_name': 'foo.txt', 'quality': 'bad'}


Delete documents with specific metadata.

In [20]:
db.delete(filter={"quality": "bad"})

# Now the similarity search with the same filter will return no results
docs = db.similarity_search("foobar", k=2, filter={"quality": "bad"})
print(len(docs))

Plain SQL Placeholder '?' for value='bad'
Plain SQL Placeholder '?' for value='bad'


0


## Advanced filtering
In addition to the basic value-based filtering capabilities, it is possible to use more advanced filtering.
The table below shows the available filter operators.

| Operator | Semantic                 |
|----------|-------------------------|
| `$eq`    | Equality (==)           |
| `$ne`    | Inequality (!=)         |
| `$lt`    | Less than (&lt;)           |
| `$lte`   | Less than or equal (&lt;=) |
| `$gt`    | Greater than (>)        |
| `$gte`   | Greater than or equal (>=) |
| `$in`    | Contained in a set of given values  (in)    |
| `$nin`   | Not contained in a set of given values  (not in)  |
| `$between` | Between the range of two boundary values |
| `$like`  | Text equality based on the "LIKE" semantics in SQL (using "%" as wildcard)  |
| `$contains` | Filters documents containing a specific keyword |
| `$and`   | Logical "and", supporting 2 or more operands |
| `$or`    | Logical "or", supporting 2 or more operands |

In [21]:
# Prepare some test documents
docs = [
    Document(
        page_content="First",
        metadata={"name": "Adam Smith", "is_active": True, "id": 1, "height": 10.0},
    ),
    Document(
        page_content="Second",
        metadata={"name": "Bob Johnson", "is_active": False, "id": 2, "height": 5.7},
    ),
    Document(
        page_content="Third",
        metadata={"name": "Jane Doe", "is_active": True, "id": 3, "height": 2.4},
    ),
]

db = HanaDB(
    connection=connection,
    embedding=embeddings,
    table_name="LANGCHAIN_DEMO_ADVANCED_FILTER",
)

# Delete already existing documents from the table
db.delete(filter={})
db.add_documents(docs)


# Helper function for printing filter results
def print_filter_result(result):
    if len(result) == 0:
        print("<empty result>")
    for doc in result:
        print(doc.metadata)

Filtering with `$ne`, `$gt`, `$gte`, `$lt`, `$lte`

In [22]:
advanced_filter = {"id": {"$ne": 1}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"id": {"$gt": 1}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"id": {"$gte": 1}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"id": {"$lt": 1}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"id": {"$lte": 1}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

Filter: {'id': {'$ne': 1}}
{'name': 'Bob Johnson', 'is_active': False, 'id': 2, 'height': 5.7}
{'name': 'Jane Doe', 'is_active': True, 'id': 3, 'height': 2.4}
Filter: {'id': {'$gt': 1}}
{'name': 'Bob Johnson', 'is_active': False, 'id': 2, 'height': 5.7}
{'name': 'Jane Doe', 'is_active': True, 'id': 3, 'height': 2.4}
Filter: {'id': {'$gte': 1}}
{'name': 'Bob Johnson', 'is_active': False, 'id': 2, 'height': 5.7}
{'name': 'Adam Smith', 'is_active': True, 'id': 1, 'height': 10.0}
{'name': 'Jane Doe', 'is_active': True, 'id': 3, 'height': 2.4}
Filter: {'id': {'$lt': 1}}
<empty result>
Filter: {'id': {'$lte': 1}}
{'name': 'Adam Smith', 'is_active': True, 'id': 1, 'height': 10.0}


Filtering with `$between`, `$in`, `$nin`

In [23]:
advanced_filter = {"id": {"$between": (1, 2)}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"name": {"$in": ["Adam Smith", "Bob Johnson"]}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"name": {"$nin": ["Adam Smith", "Bob Johnson"]}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

Filter: {'id': {'$between': (1, 2)}}
{'name': 'Bob Johnson', 'is_active': False, 'id': 2, 'height': 5.7}
{'name': 'Adam Smith', 'is_active': True, 'id': 1, 'height': 10.0}
Filter: {'name': {'$in': ['Adam Smith', 'Bob Johnson']}}


Plain SQL Placeholder '?' for value='Adam Smith'
Plain SQL Placeholder '?' for value='Bob Johnson'


{'name': 'Bob Johnson', 'is_active': False, 'id': 2, 'height': 5.7}
{'name': 'Adam Smith', 'is_active': True, 'id': 1, 'height': 10.0}
Filter: {'name': {'$nin': ['Adam Smith', 'Bob Johnson']}}


Plain SQL Placeholder '?' for value='Adam Smith'
Plain SQL Placeholder '?' for value='Bob Johnson'


{'name': 'Jane Doe', 'is_active': True, 'id': 3, 'height': 2.4}


Text filtering with `$like`

In [24]:
advanced_filter = {"name": {"$like": "a%"}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"name": {"$like": "%a%"}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

Filter: {'name': {'$like': 'a%'}}


Plain SQL Placeholder '?' for value='a%'


<empty result>
Filter: {'name': {'$like': '%a%'}}


Plain SQL Placeholder '?' for value='%a%'


{'name': 'Adam Smith', 'is_active': True, 'id': 1, 'height': 10.0}
{'name': 'Jane Doe', 'is_active': True, 'id': 3, 'height': 2.4}


Text filtering with `$contains`

In [21]:
advanced_filter = {"name": {"$contains": "bob"}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"name": {"$contains": "bo"}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"name": {"$contains": "Adam Johnson"}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"name": {"$contains": "Adam Smith"}}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

Filter: {'name': {'$contains': 'bob'}}
{'name': 'Bob Johnson', 'is_active': False, 'id': 2, 'height': 5.7}
Filter: {'name': {'$contains': 'bo'}}
<empty result>
Filter: {'name': {'$contains': 'Adam Johnson'}}
<empty result>
Filter: {'name': {'$contains': 'Adam Smith'}}
{'name': 'Adam Smith', 'is_active': True, 'id': 1, 'height': 10.0}


Combined filtering with `$and`, `$or`

In [22]:
advanced_filter = {"$or": [{"id": 1}, {"name": "bob"}]}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"$and": [{"id": 1}, {"id": 2}]}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {"$or": [{"id": 1}, {"id": 2}, {"id": 3}]}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

advanced_filter = {
    "$and": [{"name": {"$contains": "bob"}}, {"name": {"$contains": "johnson"}}]
}
print(f"Filter: {advanced_filter}")
print_filter_result(db.similarity_search("just testing", k=5, filter=advanced_filter))

Filter: {'$or': [{'id': 1}, {'name': 'bob'}]}
{'name': 'Adam Smith', 'is_active': True, 'id': 1, 'height': 10.0}
Filter: {'$and': [{'id': 1}, {'id': 2}]}
<empty result>
Filter: {'$or': [{'id': 1}, {'id': 2}, {'id': 3}]}
{'name': 'Adam Smith', 'is_active': True, 'id': 1, 'height': 10.0}
{'name': 'Jane Doe', 'is_active': True, 'id': 3, 'height': 2.4}
{'name': 'Bob Johnson', 'is_active': False, 'id': 2, 'height': 5.7}
Filter: {'$and': [{'name': {'$contains': 'bob'}}, {'name': {'$contains': 'johnson'}}]}
{'name': 'Bob Johnson', 'is_active': False, 'id': 2, 'height': 5.7}


## Using a VectorStore as a retriever in chains for retrieval augmented generation (RAG)

In [25]:
# Access the vector DB with a new table
db = HanaDB(
    connection=connection,
    embedding=embeddings,
    table_name="LANGCHAIN_DEMO_RETRIEVAL_CHAIN",
)

# Delete already existing entries from the table
db.delete(filter={})

# add the loaded document chunks from the "State Of The Union" file
db.add_documents(text_chunks)

# Create a retriever instance of the vector store
retriever = db.as_retriever()

Define the prompt.

In [26]:
from langchain_core.prompts import PromptTemplate

prompt_template = """
You are an expert in state of the union topics. You are provided multiple context items that are related to the prompt you have to answer.
Use the following pieces of context to answer the question at the end.

'''
{context}
'''

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

Create the ConversationalRetrievalChain, which handles the chat history and the retrieval of similar document chunks to be added to the prompt.

In [29]:
from gen_ai_hub.proxy.langchain.init_models import init_llm
 
llm = init_llm(model_list[0], temperature=0.0, max_tokens=256)

In [27]:
llm

ChatBedrock(client=<gen_ai_hub.proxy.native.amazon.clients.BedrockRuntime object at 0x0000017659843ED0>, model_id='anthropic.claude-3-5-sonnet-20240620-v1:0', model_kwargs={'temperature': 0.0, 'max_tokens': 256, 'top_p': 1.0}, client_params={'deployment_id': 'ddf582baaa876fa4', 'model_name': 'anthropic--claude-3.5-sonnet', 'config_id': '', 'config_name': '', 'proxy_client': GenAIHubProxyClient(base_url=None, auth_url=None, client_id=None, client_secret=None, resource_group=None, ai_core_client=<ai_core_sdk.ai_core_v2_client.AICoreV2Client object at 0x00000176207D1F10>)})

In [35]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo")
memory = ConversationBufferMemory(
    memory_key="chat_history", output_key="answer", return_messages=True
)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    db.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    memory=memory,
    verbose=False,
    combine_docs_chain_kwargs={"prompt": PROMPT},
)

Ask the first question (and verify how many text chunks have been used).

In [36]:
question = "What about Mexico and Guatemala?"

result = qa_chain.invoke({"question": question})
print("Answer from LLM:")
print("================")
print(result["answer"])

source_docs = result["source_documents"]
print("================")
print(f"Number of used source document chunks: {len(source_docs)}")

Answer from LLM:
The context highlights efforts to enhance border security and combat human trafficking through joint patrols with Mexico and Guatemala. These collaborative measures aim to catch more human traffickers and improve the overall safety and security of the region. Additionally, there is a focus on securing commitments from these countries to host more refugees and to strengthen their own borders, indicating a comprehensive approach to immigration and border management that involves cooperation with neighboring nations.
Number of used source document chunks: 5


Examine the used chunks of the chain in detail. Check if the best ranked chunk contains info about "Mexico and Guatemala" as mentioned in the question.

In [37]:
for doc in source_docs:
    print("-" * 80)
    print(doc.page_content)
    print(doc.metadata)

--------------------------------------------------------------------------------
We can do both. At our border, we’ve installed new technology like cutting-edge scanners to better detect drug smuggling.  

We’ve set up joint patrols with Mexico and Guatemala to catch more human traffickers.  

We’re putting in place dedicated immigration judges so families fleeing persecution and violence can have their cases heard faster. 

We’re securing commitments and supporting partners in South and Central America to host more refugees and secure their own borders.
{'source': '../../how_to/state_of_the_union.txt'}
--------------------------------------------------------------------------------
Officer Mora was 27 years old. 

Officer Rivera was 22. 

Both Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. 

I spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the 

Ask another question on the same conversational chain. The answer should relate to the previous answer given.

In [38]:
question = "How many casualties were reported after that? write in English."

result = qa_chain.invoke({"question": question})
print("Answer from LLM:")
print("================")
print(result["answer"])

Answer from LLM:
The provided context does not mention any casualties reported after the joint patrols with Mexico and Guatemala. It focuses on various measures taken to enhance border security and support for law enforcement, but does not provide specific information about casualties related to those patrols.


## Standard tables vs. "custom" tables with vector data

As default behaviour, the table for the embeddings is created with 3 columns:

- A column `VEC_TEXT`, which contains the text of the Document
- A column `VEC_META`, which contains the metadata of the Document
- A column `VEC_VECTOR`, which contains the embeddings-vector of the Document's text

In [39]:
# Access the vector DB with a new table
db = HanaDB(
    connection=connection, embedding=embeddings, table_name="LANGCHAIN_DEMO_NEW_TABLE"
)

# Delete already existing entries from the table
db.delete(filter={})

# Add a simple document with some metadata
docs = [
    Document(
        page_content="A simple document",
        metadata={"start": 100, "end": 150, "doc_name": "simple.txt"},
    )
]
db.add_documents(docs)

[]

Show the columns in table "LANGCHAIN_DEMO_NEW_TABLE"

In [40]:
cur = connection.cursor()
cur.execute(
    "SELECT COLUMN_NAME, DATA_TYPE_NAME FROM SYS.TABLE_COLUMNS WHERE SCHEMA_NAME = CURRENT_SCHEMA AND TABLE_NAME = 'LANGCHAIN_DEMO_NEW_TABLE'"
)
rows = cur.fetchall()
for row in rows:
    print(row)
cur.close()

('VEC_META', 'NCLOB')
('VEC_TEXT', 'NCLOB')
('VEC_VECTOR', 'REAL_VECTOR')


Show the value of the inserted document in the three columns 

In [41]:
cur = connection.cursor()
cur.execute(
    "SELECT VEC_TEXT, VEC_META, TO_NVARCHAR(VEC_VECTOR) FROM LANGCHAIN_DEMO_NEW_TABLE LIMIT 1"
)
rows = cur.fetchall()
print(rows[0][0])  # The text
print(rows[0][1])  # The metadata
print(rows[0][2])  # The vector
cur.close()

A simple document
{"start": 100, "end": 150, "doc_name": "simple.txt"}
[-0.025887018,0.037281916,0.011992734,-0.03336357,0.003929151,-0.05137068,0.0021338423,0.008355298,0.0008337291,-0.00089315255,0.009471739,-0.0316637,-0.0037022615,-0.0040047807,0.014664989,0.050592773,0.013843865,0.014038342,0.03624471,0.03316189,0.015284434,0.044110212,0.023596514,0.0070948005,0.024302393,-0.043533985,-0.03604303,-0.03503463,0.0040984177,-0.046904914,0.029704526,-0.03313308,-0.0014306648,-0.010069575,-0.0047646808,0.045233857,0.024777781,-0.0076134056,-0.05033347,-0.038491994,-0.0035582045,-0.020484887,0.052206207,-0.008369704,0.025224358,0.034573648,0.012821062,0.067245744,0.00046503358,0.04024949,-0.044167835,-0.020412859,0.010710628,0.07611965,-0.008960337,-0.038924165,-0.02525317,-0.0011002343,-0.0070083668,0.009277262,-0.0038427168,0.0015918284,0.006190844,0.053963702,-0.007800679,0.02297707,-0.045406725,0.017848646,0.03607184,0.016379265,0.022789797,0.010905105,0.0044513573,0.032038245,-0.01

Custom tables must have at least three columns that match the semantics of a standard table

- A column with type `NCLOB` or `NVARCHAR` for the text/context of the embeddings
- A column with type `NCLOB` or `NVARCHAR` for the metadata 
- A column with type `REAL_VECTOR` for the embedding vector

The table can contain additional columns. When new Documents are inserted into the table, these additional columns must allow NULL values.

In [42]:
# Create a new table "MY_OWN_TABLE_ADD" with three "standard" columns and one additional column
my_own_table_name = "MY_OWN_TABLE_ADD"
cur = connection.cursor()
cur.execute(
    (
        f"CREATE TABLE {my_own_table_name} ("
        "SOME_OTHER_COLUMN NVARCHAR(42), "
        "MY_TEXT NVARCHAR(2048), "
        "MY_METADATA NVARCHAR(1024), "
        "MY_VECTOR REAL_VECTOR )"
    )
)

# Create a HanaDB instance with the own table
db = HanaDB(
    connection=connection,
    embedding=embeddings,
    table_name=my_own_table_name,
    content_column="MY_TEXT",
    metadata_column="MY_METADATA",
    vector_column="MY_VECTOR",
)

# Add a simple document with some metadata
docs = [
    Document(
        page_content="Some other text",
        metadata={"start": 400, "end": 450, "doc_name": "other.txt"},
    )
]
db.add_documents(docs)

# Check if data has been inserted into our own table
cur.execute(f"SELECT * FROM {my_own_table_name} LIMIT 1")
rows = cur.fetchall()
print(rows[0][0])  # Value of column "SOME_OTHER_DATA". Should be NULL/None
print(rows[0][1])  # The text
print(rows[0][2])  # The metadata
print(rows[0][3])  # The vector

cur.close()

None
Some other text
{"start": 400, "end": 450, "doc_name": "other.txt"}


Add another document and perform a similarity search on the custom table.

In [43]:
docs = [
    Document(
        page_content="Some more text",
        metadata={"start": 800, "end": 950, "doc_name": "more.txt"},
    )
]
db.add_documents(docs)

query = "What's up?"
docs = db.similarity_search(query, k=2)
for doc in docs:
    print("-" * 80)
    print(doc.page_content)

--------------------------------------------------------------------------------
Some other text
--------------------------------------------------------------------------------
Some more text


### Filter Performance Optimization with Custom Columns

To allow flexible metadata values, all metadata is stored as JSON in the metadata column by default. If some of the used metadata keys and value types are known, they can be stored in additional columns instead by creating the target table with the key names as column names and passing them to the HanaDB constructor via the specific_metadata_columns list. Metadata keys that match those values are copied into the special column during insert. Filters use the special columns instead of the metadata JSON column for keys in the specific_metadata_columns list.

In [44]:
# Create a new table "PERFORMANT_CUSTOMTEXT_FILTER" with three "standard" columns and one additional column
my_own_table_name = "PERFORMANT_CUSTOMTEXT_FILTER"
cur = connection.cursor()
cur.execute(
    (
        f"CREATE TABLE {my_own_table_name} ("
        "CUSTOMTEXT NVARCHAR(500), "
        "MY_TEXT NVARCHAR(2048), "
        "MY_METADATA NVARCHAR(1024), "
        "MY_VECTOR REAL_VECTOR )"
    )
)

# Create a HanaDB instance with the own table
db = HanaDB(
    connection=connection,
    embedding=embeddings,
    table_name=my_own_table_name,
    content_column="MY_TEXT",
    metadata_column="MY_METADATA",
    vector_column="MY_VECTOR",
    specific_metadata_columns=["CUSTOMTEXT"],
)

# Add a simple document with some metadata
docs = [
    Document(
        page_content="Some other text",
        metadata={
            "start": 400,
            "end": 450,
            "doc_name": "other.txt",
            "CUSTOMTEXT": "Filters on this value are very performant",
        },
    )
]
db.add_documents(docs)

# Check if data has been inserted into our own table
cur.execute(f"SELECT * FROM {my_own_table_name} LIMIT 1")
rows = cur.fetchall()
print(
    rows[0][0]
)  # Value of column "CUSTOMTEXT". Should be "Filters on this value are very performant"
print(rows[0][1])  # The text
print(
    rows[0][2]
)  # The metadata without the "CUSTOMTEXT" data, as this is extracted into a sperate column
print(rows[0][3])  # The vector

cur.close()

Filters on this value are very performant
Some other text
{"start": 400, "end": 450, "doc_name": "other.txt", "CUSTOMTEXT": "Filters on this value are very performant"}


The special columns are completely transparent to the rest of the langchain interface. Everything works as it did before, just more performant.

In [46]:
docs = [
    Document(
        page_content="Some more text football",
        metadata={
            "start": 800,
            "end": 950,
            "doc_name": "more.txt",
            "CUSTOMTEXT": "Another customtext value football",
        },
    )
]
db.add_documents(docs)



[]

In [47]:
advanced_filter = {"CUSTOMTEXT": {"$like": "%foot%"}}
query = "What's up?"
docs = db.similarity_search(query, k=2, filter=advanced_filter)
for doc in docs:
    print("-" * 80)
    print(doc.page_content)

Plain SQL Placeholder '?' for value='%foot%'


--------------------------------------------------------------------------------
Some more text football


In [51]:
advanced_filter = {"CUSTOMTEXT": {"$like": "%foot%"}}
query = "What's up?"
docs = db.similarity_search(query, k=2, filter=advanced_filter)
for doc in docs:
    print("-" * 80)
    print(doc.page_content)

Plain SQL Placeholder '?' for value='%foot%'


--------------------------------------------------------------------------------
Some more text football


In [50]:
advanced_filter = {"CUSTOMTEXT": {"$like": "%football%"}}
query = "What's up?"
docs = db.similarity_search(query, k=2,)
for doc in docs:
    print("-" * 80)
    print(doc.page_content)

--------------------------------------------------------------------------------
Some other text
--------------------------------------------------------------------------------
Some more text
